# Housing

In [1]:
import sys
sys.path.append('../src')

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

### Homeownership rate

In [2]:
data = fred_df('RHORUSQ156N')['VALUE']
data.to_csv(data_dir / 'homeown.csv', index_label='date', header=True)
node = end_node(data, 'violet!60!magenta', percent=True, 
                date='q', full_year=True)
write_txt(text_dir / 'homeown_node.txt', node)
datelt = dtxt(data.index[-1])['qtr1']
latest = data.iloc[-1]
ch3yr = value_text(data.diff(12).iloc[-1], ptype='pp', adj='total')
text = (f'As of {datelt}, the Census Bureau '+
        '\href{https://www.census.gov/housing/hvs/index.html}{report} '+
        f'a homeownership rate of {latest:.1f} percent '+
        '(see {\color{violet!60!magenta}\\textbf{---}}). Over the '+
        f'past three years, the overall US homeownership rate {ch3yr}.')
write_txt(text_dir / 'homeown.txt', text)
print(text)

As of 2021 Q4, the Census Bureau \href{https://www.census.gov/housing/hvs/index.html}{report} a homeownership rate of 65.5 percent (see {\color{violet!60!magenta}\textbf{---}}). Over the past three years, the overall US homeownership rate increased by a total of 0.7 percentage point.


### By Race/Ethnicity

In [3]:
url = 'https://www.census.gov/housing/hvs/data/histtab16.xlsx'

data = pd.read_excel(url)
df = data.iloc[7:-9, :-6].reset_index().drop('index', axis=1)
df.columns = ['Date', 'Total', 'White', 'Black', 'Other', 'Other2', 'Hispanic']
df = df.replace('2002r', 2002).replace('20033', 2003).dropna(how='all')

q = {1: '01', 2: '04', 3: '07', 4: '10'}

d = {}
for i, row in df.iterrows():
    if i not in d.keys():
        d[i] = {}
    if type(row['Date']) == int:
        for y in list(range(i, i+5)):
            if y in df.index:
                d[y] = {'quarter': ''}
                d[y]['year'] = row['Date']
    if (type(row['Date']) != int) & ('.' in str(row['Date'])):
        d[i]['quarter'] = q[int(row['Date'][0])]
        
    if 'year' not in d[i].keys():
        d[i] = {'date': '', 'year': '', 'quarter': ''}
    else:
        d[i]['date'] = f'{d[i]["year"]}-{d[i]["quarter"]}-01'
        
dates = {k: v['date'] for k, v in d.items()}
df['date'] = df.index.map(dates)
df = df.dropna()
df = df.set_index(pd.to_datetime(df.date))
result = df[['White', 'Black', 'Hispanic', 'Other', 'Total']]
result.to_csv(data_dir / 'homeown2.csv', index_label='date', header=True)

color_list = [('Black', 'red!20!orange!95!black'), 
              ('White', 'blue!80!cyan!80!black'),
              ('Hispanic', 'green!80!blue!85!black'),
              ('Other', 'red!80!black')]
for group, color in color_list:
    write_txt(text_dir / f'homeown_{group.lower()}_node.txt', 
              end_node(result[group], color, percent=True))

wtlt = result.White.iloc[-1]
blmax = result['Black'].max()
blidxmax = dtxt(result['Black'].idxmax())['qtr2']
blmin = result.loc['2006':, 'Black'].min()
blidxmin = dtxt(result.loc['2006':, 'Black'].idxmin())['qtr1']
bllt = result.Black.iloc[-1]
ltdt = dtxt(result.index[-1])['qtr1']
hsmax = result['Hispanic'].max()
hsidxmax = dtxt(result['Hispanic'].idxmax())['qtr2']
hslt = result['Hispanic'].iloc[-1]
compare = compare_text(hslt, hsmax, [0.3, 1.0, 3.0])
if hslt == hsmax:
    cmp_txt = 'the highest on record'
else:
    cmp_txt = f'{compare} the {hsmax:.1f} percent peak rate in {hsidxmax}'

text = (f'Around three-quarters ({wtlt:.1f} percent in {ltdt}) of non-Hispanic '+
        'white households own their home (see {\color{blue!80!cyan!80!black}\\textbf{---}}), '+
        'compared to fewer than half of black and Hispanic households. ')
write_txt(text_dir / 'homeown2.txt', text)
print(text, '\n')

txt2 = ('During the housing bubble, the homeownership rate for black households increased '+
        f'by nearly ten percentage points, peaked at {blmax:.1f} percent '+
        f'in {blidxmax}, and fell to {blmin:.1f} percent in {blidxmin}. The current '+
        f'homeownership rate for black households is {bllt:.1f} percent, as of '+
        f'{ltdt} (see {{\color{{red!20!orange!95!black}}\\textbf{{---}}}}). '+
        'The rate for Hispanic households '+
        f'of any race is {hslt:.1f} percent in {ltdt}, {cmp_txt} '+
        '(see {\color{green!80!blue!85!black}\\textbf{---}}). ')
write_txt(text_dir / 'homeown3.txt', txt2)
print(txt2)

Around three-quarters (74.4 percent in 2021 Q4) of non-Hispanic white households own their home (see {\color{blue!80!cyan!80!black}\textbf{---}}), compared to fewer than half of black and Hispanic households.  

During the housing bubble, the homeownership rate for black households increased by nearly ten percentage points, peaked at 49.7 percent in the second quarter of 2004, and fell to 40.6 percent in 2019 Q2. The current homeownership rate for black households is 43.1 percent, as of 2021 Q4 (see {\color{red!20!orange!95!black}\textbf{---}}). The rate for Hispanic households of any race is 48.4 percent in 2021 Q4, substantially below the 51.4 percent peak rate in the second quarter of 2020 (see {\color{green!80!blue!85!black}\textbf{---}}). 


### FHFA Housing Price Index

In [4]:
df = pd.read_csv('https://www.fhfa.gov/HPI_master.csv')
srs = ['place_name', 'yr', 'period', 'index_sa']
dr = (df.loc[(df['level'] == 'USA or Census Division') & 
             (df['frequency'] == 'monthly'), srs]
        .rename({'yr': 'year', 'period': 'month'}, axis=1)
        .assign(day = 1))

data = (dr.set_index([pd.to_datetime(dr[['year', 'month', 'day']]), 
                   dr['place_name']])
          .index_sa.unstack().pct_change(12) * 100).dropna()
data.columns = [i.replace(' Division', '') for i in data.columns]
ch = data['United States']
ch.to_csv(data_dir / 'hpi.csv', index_label='date', header='True')
node = end_node(ch, 'blue!40!cyan', date='m', 
                full_year=True, percent=True)
write_txt(text_dir / 'fhfa_node.txt', node)

chtxt = value_text(ch.iloc[-1])
ltdt = dtxt(ch.index[-1])['mon1']
div = data.iloc[-1].idxmax()
maxval = data.iloc[-1].max()
divs = pd.read_csv('raw/census_regions.csv')
div_list = divs[divs['Division'] == div].State.to_list()
states = f"{', '.join(div_list[:-1])}, and {div_list[-1]}"

text = (f'{chtxt} over the year ending {ltdt}. '+
          'Among Census Divisions, the fastest one-year housing '+
         f'price index growth rate in {ltdt} was {maxval:.1f} '+
         f'percent in the {div} Division, which includes {states}.')
write_txt(text_dir / 'fhfa.txt', text)
print(text)

increased 17.5 percent over the year ending November 2021. Among Census Divisions, the fastest one-year housing price index growth rate in November 2021 was 22.8 percent in the Mountain Division, which includes Arizona, Colorado, Idaho, Montana, New Mexico, Nevada, Utah, and Wyoming.


In [5]:
t = pd.DataFrame()
for i in [-1, -2, -3, -4, -13, -25, -37]:
    t[dtxt(data.index[i])['mon6']] = data.iloc[i]
    
t['`03--`05 Average'] = data.loc['2003':'2005'].mean()
t['`09--`12 Average'] = data.loc['2009':'2012'].mean()

final = (t.rename({'United States': '\\textbf{United States}'})
          .sort_values(dtxt(data.index[-1])['mon6'], ascending=False)
          .round(1))

final.to_csv(data_dir / 'hpi.tex', sep='&', line_terminator='\\\ ', quotechar=' ')
final

,Nov `21,Oct `21,Sep `21,Aug `21,Nov `20,Nov `19,Nov `18,`03--`05 Average,`09--`12 Average
Mountain,22.8,23.3,23.9,25.7,13.9,6.7,7.8,11.1,-4.2
South Atlantic,20.4,19.5,19.9,20.7,11.6,5.5,6.1,11.3,-3.7
East South Central,19.8,18.9,17.6,18.8,11.7,4.6,6.5,5.1,-1.6
Pacific,18.9,20.0,19.7,21.3,12.1,5.1,5.2,18.4,-3.9
West South Central,18.1,17.1,18.1,16.9,9.0,3.8,4.5,4.3,0.3
\textbf{United States},17.5,17.4,17.8,18.6,11.2,5.2,5.7,9.2,-2.5
New England,15.4,15.6,18.3,18.7,12.9,5.1,4.7,10.3,-2.2
Middle Atlantic,14.0,14.2,14.3,15.6,11.1,4.5,5.3,11.3,-2.3
East North Central,13.9,14.0,14.7,15.2,10.6,5.9,5.7,4.3,-2.4
West North Central,13.3,13.5,13.9,14.9,10.0,5.1,5.1,5.4,-1.1


### Ratio of price to rent

In [6]:
hpi = (dr.set_index(pd.to_datetime(dr[['year', 'month', 'day']]))
         .query('place_name == "United States"').index_sa)

rent = fred_df('CUUR0000SEHA')['VALUE']

ratio = (hpi / rent).dropna()
ratio = ratio / ratio.iloc[0]
ratio.name = 'VALUE'

ratio.to_csv(data_dir / 'hpi_rent_ratio.csv', index_label='date', header=True)

ltdate = dtxt(ratio.index[-1])['mon1']
ltval = (ratio.iloc[-1] - 1) * 100
text = (f'As of {ltdate}, housing prices are {ltval:.1f} percent '+
        'above the rental equivalent (see {\color{red}\\textbf{---}}).')
write_txt(text_dir / 'hp_rent.txt', text)
print(text)

As of November 2021, housing prices are 44.4 percent above the rental equivalent (see {\color{red}\textbf{---}}).


### Owner's equity share in real estate

In [7]:
data = fred_df('HOEREPHRE')['VALUE'].astype('float')
data.to_csv(data_dir / 'homeeq.csv', index_label='date', header=True)

datelt = dtxt(data.index[-1])['qtr1']
latest = data.iloc[-1]
ch3, ch1 = [value_text(data.diff(i).iloc[-1], style='increase_by', 
                   adj='total', ptype='pp') for i in [12, 4]]
avg89 = data.loc['1989'].mean()
compare = compare_text(latest, avg89, [0.05, 1.0, 5.0])
node_color = 'blue!80!violet'
node = end_node(data, node_color, percent=True)
write_txt(text_dir / 'homeeq_node.txt', node)

fedlink = 'https://www.federalreserve.gov/releases/z1/20191212/html/b101.htm'
text = (f'As of {datelt}, the Federal Reserve \href{{fedlink}}{{report}} '+
        f"owners\' equity is {latest:.1f} percent of residential "+
        f'real estate (see {{\color{{{node_color}}}\\textbf{{---}}}}). '+
        f"Over the past three years, the owners\' equity share {ch3}. "+
        f'Over the past year, the share {ch1}. The current share is {compare} '+
        f'the 1989 average of {avg89:.1f} percent.')
write_txt(text_dir / 'homeeq.txt', text)
print(text)

As of 2021 Q3, the Federal Reserve \href{fedlink}{report} owners' equity is 68.8 percent of residential real estate (see {\color{blue!80!violet}\textbf{---}}). Over the past three years, the owners' equity share increased by a total of 4.7 percentage points. Over the past year, the share increased by a total of 2.4 percentage points. The current share is slightly above the 1989 average of 67.9 percent.


### Residential construction

In [8]:
key = census_key
base = 'https://api.census.gov/data/timeseries/eits/resconst/'
param = 'cell_value,time_slot_id,category_code,data_type_code'
t = '&time=from+1989'
oth = '&for=us&seasonally_adj=yes'

url = f'{base}?get={param}&key={key}{t}{oth}'

r = requests.get(url).json()

In [9]:
for t in ['total', 'single', 'multi']:
    df = pd.DataFrame()
    for series in ['APERMITS', 'ASTARTS', 'ACOMPLETIONS']:
        df[series] = pd.Series(
            {pd.to_datetime(i[4]): 
             float(i[0]) for i in r[1:] if i[2] == series and i[3] == t.upper()}
        ).sort_index()
    df.to_csv(data_dir / f'permits_{t}.csv', index_label='date')

In [10]:
df = pd.read_csv(data_dir / 'permits_total.csv', 
                 parse_dates=True, index_col='date')
s = series_info(df['APERMITS'])
s['val_5yr_ago'] = df['APERMITS'].iloc[-61]

if s['days_since_match'] > 100:
    hlt = f", {s['last_matched']}"
else:
    hlt = ''
month = s['date_latest'].strftime('%B')

d = {}
for i in ['val_prev', 'val_year_ago', 'val_5yr_ago']:
    mo_ch = s["val_latest"] - s[i]
    mo_pch = (s["val_latest"] / s[i] - 1) * 100
    if mo_ch >= 0.1:
        txt = f'increased by {abs(mo_ch)*1000:,.0f} ({mo_pch:.1f} percent)'
    elif mo_ch <= -0.1:
        txt = f'decreased by {abs(mo_ch)*1000:,.0f} ({mo_pch:.1f} percent)'
    else:
        txt = 'were virtually unchanged' 
    d[i] = txt

date = s["date_latest_ft"]    
    
text = (f'In {date}, a seasonally-adjusted annual rate of '+
        f'{s["val_latest"]*1000:,.0f} new '+
        f'residential housing units were authorized by building permits{hlt} '+
        '(see {\color{blue!70!green}\\textbf{---}}). '+
        f'Permits issued {d["val_prev"]} over the previous month, '+
        f'{d["val_year_ago"]} over last {month}, and '+
        f'{d["val_5yr_ago"]} total over the past five years.')
write_txt(text_dir / 'permits.txt', text)
print(text)

prdate = dtxt(df.index[-2])['mon3']
pryrdate = dtxt(df.index[-13])['mon1']
ltval = df['ACOMPLETIONS'].iloc[-1]
prval = df['ACOMPLETIONS'].iloc[-2]
pryrval = df['ACOMPLETIONS'].iloc[-3]
text = (f'In {date}, a seasonally-adjusted annual rate of '+
        f'{ltval*1000:,.0f} new residential units were completed '+
        '(see {\color{blue!30!cyan}\\textbf{---}}), compared to '+
        f'{prval*1000:,.0f} in {prdate} and {pryrval*1000:,.0f} in {pryrdate}.')
write_txt(text_dir / 'completions.txt', text)
print(text)

In December 2021, a seasonally-adjusted annual rate of 1,885,000 new residential housing units were authorized by building permits, the highest level since May 2006 (see {\color{blue!70!green}\textbf{---}}). Permits issued increased by 168,000 (9.8 percent) over the previous month, increased by 127,000 (7.2 percent) over last December, and increased by 633,000 (50.6 percent) total over the past five years.
In December 2021, a seasonally-adjusted annual rate of 1,295,000 new residential units were completed (see {\color{blue!30!cyan}\textbf{---}}), compared to 1,418,000 in November and 1,255,000 in December 2020.


In [11]:
for t in ['single', 'multi']:
    df = pd.read_csv(data_dir / f'permits_{t}.csv', 
                     parse_dates=True, index_col='date')
    date = dtxt(df.index[-1])['mon1']
    pval = df['APERMITS'].iloc[-1]
    cval = df['ACOMPLETIONS'].iloc[-1]
    text = (f'In {date}, a seasonally-adjusted annual rate of '+
            f'{pval*1000:,.0f} new {t}-family residential units were permitted '+
            f'and {cval*1000:,.0f} were completed.')
    write_txt(text_dir / f'permits_{t}.txt', text)
    print(text)

In December 2021, a seasonally-adjusted annual rate of 1,128,000 new single-family residential units were permitted and 990,000 were completed.
In December 2021, a seasonally-adjusted annual rate of 690,000 new multi-family residential units were permitted and 299,000 were completed.


### Household formation estimates

In [4]:
files = ['histtab8.xlsx', 'hist_tab_8a_v2020.xlsx']
url = 'https://www.census.gov/housing/hvs/data/'

results = {}

for file in files:
    data = pd.read_excel(url + file).iloc[4:, :5]
    data.columns = ['category', 'Q1', 'Q2', 'Q3', 'Q4']
    years = []
    for h in data[data['Q1'] == '1st Qtr'].index - 1:
        year_raw = data.loc[h, 'Q1']
        if type(year_raw) == int:
            year = year_raw
        elif type(year_raw) == str:
            year = int(year_raw[:4])
        elif type(year) == float:
            year = year + 1
        years.append(year)
    data.loc[data['Q1'] == '1st Qtr', 'category'] = years
    data = data.dropna(subset=['category'])
    
    df = pd.DataFrame()
    levels = pd.DataFrame()
    for series in ['Owner', 'Renter']:
        srs = data.loc[data['category'].astype('str').str.contains(series)]
        srs.index = years
        srs = srs[['Q1', 'Q2', 'Q3', 'Q4']].unstack().swaplevel()
        srs.index = pd.to_datetime([f'{i[0]}-{i[1]}' for i in srs.index])
        srs = srs.sort_index()
        df[series] = srs
        levels[series] = srs
    df = df.dropna()
    df['Total'] = df.sum(axis=1)
    
    # Convert to share of total change
    for s in ['Owner', 'Renter']:  
        df[s] = ((df[s] - df[s].shift(4))
                  /df['Total'].shift(4)) * 100
        
    df = (df.reset_index()
            .drop_duplicates(subset='index', keep='last')
            .set_index('index'))
    
    results[file] = df[['Owner', 'Renter']].dropna().rolling(4).mean().loc['1989':]

In [5]:
final = results['histtab8.xlsx'].loc[:'2001'].append(results['hist_tab_8a_v2020.xlsx'].loc['2002':])
final['pop'] = (nipa_df(retrieve_table('T70100')['Data'], ['B230RC'])
                .pct_change(4).dropna() * 100)

final.to_csv(data_dir / 'hhform.csv', index_label='date')

levels = levels.dropna()
ldate = f'{levels.index[-1].year} Q{levels.index[-1].quarter}'
pdate = f'{levels.index[-2].year} Q{levels.index[-2].quarter}'
tot = levels.iloc[-1].sum() / 1000
rtot = levels['Renter'].iloc[-1] / 1000
rsh = rtot / tot * 100
otot = levels['Owner'].iloc[-1] / 1000
osh = otot / tot * 100

ch = levels.diff(4).rolling(4).mean().iloc[-1]
incdec = ['increase' if ch.sum() >= 0 else 'decrease']
chtot = [f'{abs(ch.sum()) / 1000:.1f} million' if ch.sum() > 1000 else f'{abs(ch.sum()):.0f} thousand'][0]

t = {name: [f'{abs(s) / 1000:.1f} million {["net new" if s >= 0 else "net fewer"][0]}' 
            if s > 1000 
            else f'{abs(s):.0f},000 {["net new" if s >= 0 else "net fewer"][0]}'][0]
     for name, s in [('tot', ch.sum()), ('rent', ch.Renter), ('own', ch.Owner)]}

text = (f'As of {ldate}, there are {tot:.1f} million total occupied '+
        f'housing units in the US, of which {rtot:.1f} million ({rsh:.1f} percent) '+
        f'are rented, and {otot:.1f} million ({osh:.1f} percent) are '+
        'owner-occupied. There was an average annual net total '+
        f'{incdec[0]} of {chtot} housing units '+
        f'over the year ending {ldate}, the result of {t["rent"]} '+
        f'renter households and {t["own"]} '+
        f'owner-occupied households. ')

In [6]:
totgr = final[['Owner', 'Renter']].sum(axis=1).iloc[-1]
ogr = final['Owner'].iloc[-1]
rgr = final['Renter'].iloc[-1]

final['Total'] = final[['Owner', 'Renter']].sum(axis=1)

t2 = {name: [f"{['increased' if s >= 0 else 'decreased'][0]} by {abs(s):.1f} percent"][0] 
      for name, s in final.iloc[-1].iteritems()}

t3 = {name: [f"{['an increase' if s >= 0 else 'a decreased'][0]} of {abs(s):.1f} percent"][0] 
      for name, s in final.iloc[-2].iteritems()}

t4 = {name: [f"{['a contribution' if s >= 0 else 'a reduction'][0]} of {abs(s):.1f} percent"][0] 
      for name, s in final.iloc[-1].iteritems()}

t5 = {name: [f"{[f'contributed {abs(s):.1f} percent to' if s >= 0 else f'subtracted {abs(s):.1f} percent from'][0]}"][0] 
      for name, s in final.iloc[-1].iteritems()}

text2 = (f'Over the year ending {ldate}, the total number of occupied housing units '+
         f'{t2["Total"]}, compared to {t3["Total"]} in {pdate}. Owner-occupied '+
         f'units {t5["Owner"]} total household formation on average over the year '+
         '(see\cbox{yellow!60!orange}), compared to '+
         f'{t4["Renter"]} from rented units (see\cbox{{magenta!90!blue}}).')
txt = text + text2
write_txt(text_dir / 'hhform1.txt', txt)
print(txt)

As of 2021 Q4, there are 127.4 million total occupied housing units in the US, of which 44.0 million (34.5 percent) are rented, and 83.5 million (65.5 percent) are owner-occupied. There was an average annual net total increase of 765 thousand housing units over the year ending 2021 Q4, the result of 1.7 million net new renter households and 963,000 net fewer owner-occupied households. Over the year ending 2021 Q4, the total number of occupied housing units increased by 0.6 percent, compared to an increase of 0.7 percent in 2021 Q3. Owner-occupied units subtracted 0.8 percent from total household formation on average over the year (see\cbox{yellow!60!orange}), compared to a contribution of 1.4 percent from rented units (see\cbox{magenta!90!blue}).


### New Home Sales

In [15]:
data = fred_df('HSN1F')['VALUE']
data.to_csv(data_dir / 'nhs.csv', index_label='date', header=True)

node_color = 'green!80!blue'

datelt = dtxt(data.index[-1])['mon1']
latest = data.iloc[-1] * 1000
ch1yr = data.pct_change(12).iloc[-1]

txt = value_text(ch1yr)
pc = data.loc['2020-02-01']
pcch = ((data.iloc[-1] / pc) - 1) * 100
pcvt = value_text(pcch)
text = (f'In {datelt}, the Census Bureau '+
        '\href{https://www.census.gov/construction/nrs/pdf/newressales.pdf}'+
        '{report} seasonally-adjusted single family new homes sales totaling '+
        f'{latest:,.0f} (see {{\color{{{node_color}}}\\textbf{{---}}}}). '+
        f'Over the past year, new homes sales {txt}. Pre-COVID, in '+
        'February 2020, the annualized rate of '+
        f'single family new home sales was {pc * 1000:,.0f}. '+
        f'Since February 2020, new home sales have {pcvt}.')
write_txt(text_dir / 'nhs.txt', text)
print(text)

node = end_node(data, node_color, date='m', digits=0, full_year=True)
write_txt(text_dir / 'nhs_node.txt', node)

In December 2021, the Census Bureau \href{https://www.census.gov/construction/nrs/pdf/newressales.pdf}{report} seasonally-adjusted single family new homes sales totaling 811,000 (see {\color{green!80!blue}\textbf{---}}). Over the past year, new homes sales decreased 0.1 percent. Pre-COVID, in February 2020, the annualized rate of single family new home sales was 730,000. Since February 2020, new home sales have increased 11.1 percent.
